In [ ]:
#| hide
from musy.core import *

# musy

> Python toolbox for analyzing, creating and visualizing music.

# Installation

```sh
pip install musy
```

# How to use

## Note

In [ ]:
from musy import Note, Chord, Scale

The `Note` is the basic building block from which you can create chords and songs. 

In [ ]:
c_sharp = Note("C#")
c_sharp

musy.core.Note(note='C#', oct=4)

Notes can be added and subtracted to form new notes. Each added integer represents a semitone.

In [ ]:
c_sharp + 1

musy.core.Note(note='D', oct=4)

In [ ]:
c_sharp - 1

musy.core.Note(note='C', oct=4)

In [ ]:
c_sharp + 14

musy.core.Note(note='D#', oct=5)

Intervals can be obtained by comparing a note with a different note or string.

In [ ]:
c_sharp.interval("F#")

'perfect fourth'

Shorthand is also available for intervals.

In [ ]:
c_sharp.interval("B", short=True)

'b7'

Notes can be converted to its relative major or minor.

In [ ]:
Note("C").minor()

musy.core.Note(note='A', oct=4)

In [ ]:
Note("C#").major()

musy.core.Note(note='E', oct=4)

# Chord

The `Chord` is a collection of `Note` objects played together.

In [ ]:
c_major = Chord(["C", "E", "G"])
c_major

Chord: 'C major triad'. Notes: ['C4', 'E4', 'G4']

`Chord` objects can be initialized from shorthand notation.

In [ ]:
cmaj7 = Chord.from_short("Cmaj7")
cmaj7

Chord: 'C major seventh'. Notes: ['C4', 'E4', 'G4', 'B4']

Like `Note` objects, `Chord` objects can be added and subtracted to transpose them.

In [ ]:
cmaj7 + 2

Chord: 'D major seventh'. Notes: ['D4', 'F#4', 'A4', 'C#5']

# Scale

`Scale` objects are collections of intervals from which we can generate notes and chords around a root note.

In [ ]:
dorian = Scale("dorian")
dorian

Dorian. ['1', '2', 'b3', '4', '5', '6', 'b7']

When given a root note, `Scale` generates the notes of the scale.

In [ ]:
dorian.get_notes("C")

[musy.core.Note(note='C', oct=4),
 musy.core.Note(note='D', oct=4),
 musy.core.Note(note='Eb', oct=4),
 musy.core.Note(note='F', oct=4),
 musy.core.Note(note='G', oct=4),
 musy.core.Note(note='A', oct=4),
 musy.core.Note(note='Bb', oct=4)]

Intervals can be obtained.

In [ ]:
dorian.get_interval_names()

['unison',
 'major second',
 'minor third',
 'perfect fourth',
 'perfect fifth',
 'major sixth',
 'minor seventh']

Triads and seventh chords in the scale can be generated around a root note.

In [ ]:
dorian.get_triads("D")

[Chord: 'D minor triad'. Notes: ['D4', 'F4', 'A4'],
 Chord: 'E minor triad'. Notes: ['E4', 'G4', 'B4'],
 Chord: 'F major triad'. Notes: ['F4', 'A4', 'C4'],
 Chord: 'G major triad'. Notes: ['G4', 'B4', 'D5'],
 Chord: 'A minor triad'. Notes: ['A4', 'C4', 'E5'],
 Chord: 'B diminished triad'. Notes: ['B4', 'D5', 'F5'],
 Chord: 'C major triad'. Notes: ['C5', 'E6', 'G6']]

In [ ]:
dorian.get_sevenths("E")

[Chord: 'E minor seventh'. Notes: ['E4', 'G4', 'B4', 'D4'],
 Chord: 'F# minor seventh'. Notes: ['F#4', 'A4', 'C#4', 'E5'],
 Chord: 'G major seventh'. Notes: ['G4', 'B4', 'D4', 'F#5'],
 Chord: 'A dominant seventh'. Notes: ['A4', 'C#4', 'E5', 'G5'],
 Chord: 'B minor seventh'. Notes: ['B4', 'D4', 'F#5', 'A5'],
 Chord: 'C# half diminished seventh'. Notes: ['C#5', 'E6', 'G6', 'B6'],
 Chord: 'D major seventh'. Notes: ['D5', 'F#6', 'A6', 'C#6']]

Consult `Scale.available_scales` for a list of available scales. If a scale is not available, you can create your own scale from intervals.

In [ ]:
persian = Scale.from_intervals("persian", ["1", "b2", "3", "4", "b5", "b6", "7"])
persian

Persian. ['1', 'b2', '3', '4', 'b5', 'b6', '7']

In [ ]:
persian.get_notes("C")

[musy.core.Note(note='C', oct=4),
 musy.core.Note(note='Db', oct=4),
 musy.core.Note(note='E', oct=4),
 musy.core.Note(note='F', oct=4),
 musy.core.Note(note='F#', oct=4),
 musy.core.Note(note='Ab', oct=4),
 musy.core.Note(note='B', oct=4)]